
## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [**infinite monkeys typing for an infinite amount of time**](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of William Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

We will focus specifically on Shakespeare's Sonnets in order to improve our model's ability to learn from the data.

In [1]:
import random
import sys
import os

import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import LambdaCallback

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional
from tensorflow.keras.layers import LSTM
from tensorflow.python.keras.layers import Dropout

%matplotlib inline

# a custom data prep class that we'll be using 
from data_cleaning_toolkit_class import data_cleaning_toolkit

### Use request to pull data from a URL

[**Read through the request documentation**](https://requests.readthedocs.io/en/master/user/quickstart/#make-a-request) in order to learn how to download the Shakespeare Sonnets from the Gutenberg website. 

**Protip:** Do not over think it.

In [2]:
# download all of Shakespears Sonnets from the Project Gutenberg website

# here's the link for the sonnets
url_shakespeare_sonnets = "https://www.gutenberg.org/cache/epub/1041/pg1041.txt"

# use request and the url to download all of the sonnets - save the result to `r`
r = requests.get(url_shakespeare_sonnets)


In [3]:
# move the downloaded text out of the request object - save the result to `raw_text_data`
# hint: take at look at the attributes of `r`
raw_text_data = r.text

In [4]:
# check the data type of `raw_text_data`
type(raw_text_data)

str

### Data Cleaning

In [5]:
# as usual, we are tasked with cleaning up messy data
# Question: Do you see any characters that we could use to split up the text?
raw_text_data[:3000]

"\ufeffThe Project Gutenberg EBook of Shakespeare's Sonnets, by William Shakespeare\r\n\r\nThis eBook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  You may copy it, give it away or\r\nre-use it under the terms of the Project Gutenberg License included\r\nwith this eBook or online at www.gutenberg.org\r\n\r\n\r\nTitle: Shakespeare's Sonnets\r\n\r\nAuthor: William Shakespeare\r\n\r\nPosting Date: April 7, 2014 [EBook #1041]\r\nRelease Date: September, 1997\r\nLast Updated: March 10, 2010\r\n\r\nLanguage: English\r\n\r\n\r\n*** START OF THIS PROJECT GUTENBERG EBOOK SHAKESPEARE'S SONNETS ***\r\n\r\n\r\n\r\n\r\nProduced by Joseph S. Miller and Embry-Riddle Aeronautical\r\nUniversity Library. HTML version by Al Haines.\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nTHE SONNETS\r\n\r\nby William Shakespeare\r\n\r\n\r\n\r\n\r\n  I\r\n\r\n  From fairest creatures we desire increase,\r\n  That thereby beauty's rose might never die,\r\n  But as the riper

In [6]:
# split the text into lines and save the result to `split_data`
split_data = raw_text_data.split('\r\n')

In [7]:
# we need to drop all the boilder plate text (i.e. titles and descriptions) as well as white spaces
# so that we are left with only the sonnets themselves 
split_data[40:45]

['', '', '', '  I', '']

**Use list index slicing in order to remove the titles and descriptions so we are only left with the sonnets.**


In [8]:
# sonnets exists between these indicies 
# titles and descriptions exist outside of these indicies

# use index slicing to isolate the sonnet lines - save the result to `sonnets`

# YOUR CODE HERE
sonnets = split_data[45:]
sonnets

['  From fairest creatures we desire increase,',
 "  That thereby beauty's rose might never die,",
 '  But as the riper should by time decease,',
 '  His tender heir might bear his memory:',
 '  But thou, contracted to thine own bright eyes,',
 "  Feed'st thy light's flame with self-substantial fuel,",
 '  Making a famine where abundance lies,',
 '  Thy self thy foe, to thy sweet self too cruel:',
 "  Thou that art now the world's fresh ornament,",
 '  And only herald to the gaudy spring,',
 '  Within thine own bud buriest thy content,',
 "  And tender churl mak'st waste in niggarding:",
 '    Pity the world, or else this glutton be,',
 "    To eat the world's due, by the grave and thee.",
 '',
 '  II',
 '',
 '  When forty winters shall besiege thy brow,',
 "  And dig deep trenches in thy beauty's field,",
 "  Thy youth's proud livery so gazed on now,",
 "  Will be a tatter'd weed of small worth held:",
 '  Then being asked, where all thy beauty lies,',
 '  Where all the treasure of th

In [9]:
# notice how all non-sonnet lines have far less characters than the actual sonnet lines?
# well, let's use that observation to filter out all the non-sonnet lines


In [10]:
# any string with less than n_chars characters will be filtered out - save results to `filtered_sonnets`

# YOUR CODE HERE
filtered_sonnets = []
n_chars = 56
for i in range(len(sonnets)):
    if len(sonnets[i]) < n_chars:
        filtered_sonnets.append(sonnets[i])

filtered_sonnets

['  From fairest creatures we desire increase,',
 "  That thereby beauty's rose might never die,",
 '  But as the riper should by time decease,',
 '  His tender heir might bear his memory:',
 '  But thou, contracted to thine own bright eyes,',
 "  Feed'st thy light's flame with self-substantial fuel,",
 '  Making a famine where abundance lies,',
 '  Thy self thy foe, to thy sweet self too cruel:',
 "  Thou that art now the world's fresh ornament,",
 '  And only herald to the gaudy spring,',
 '  Within thine own bud buriest thy content,',
 "  And tender churl mak'st waste in niggarding:",
 '    Pity the world, or else this glutton be,',
 "    To eat the world's due, by the grave and thee.",
 '',
 '  II',
 '',
 '  When forty winters shall besiege thy brow,',
 "  And dig deep trenches in thy beauty's field,",
 "  Thy youth's proud livery so gazed on now,",
 "  Will be a tatter'd weed of small worth held:",
 '  Then being asked, where all thy beauty lies,',
 '  Where all the treasure of th

In [11]:
# ok - much better!
# but we still need to remove all the punctuation and case normalize the text

### Use custom data cleaning tool 

Use one of the methods in `data_cleaning_toolkit` to clean your data.

There is an example of this in the guided project.

In [12]:
# instantiate the data_cleaning_toolkit class - save result to `dctk`

# YOUR CODE HERE
dctk = data_cleaning_toolkit()


In [13]:
filtered_sonnets = ''.join(filtered_sonnets)
filtered_sonnets

"  From fairest creatures we desire increase,  That thereby beauty's rose might never die,  But as the riper should by time decease,  His tender heir might bear his memory:  But thou, contracted to thine own bright eyes,  Feed'st thy light's flame with self-substantial fuel,  Making a famine where abundance lies,  Thy self thy foe, to thy sweet self too cruel:  Thou that art now the world's fresh ornament,  And only herald to the gaudy spring,  Within thine own bud buriest thy content,  And tender churl mak'st waste in niggarding:    Pity the world, or else this glutton be,    To eat the world's due, by the grave and thee.  II  When forty winters shall besiege thy brow,  And dig deep trenches in thy beauty's field,  Thy youth's proud livery so gazed on now,  Will be a tatter'd weed of small worth held:  Then being asked, where all thy beauty lies,  Where all the treasure of thy lusty days;  To say, within thine own deep sunken eyes,  Were an all-eating shame, and thriftless praise.  Ho

In [14]:
# use data_cleaning_toolkit to remove punctuation and to case normalize - save results to `clean_sonnets`
clean_sonnets = []
# YOUR CODE HERE
# raise NotImplementedError()
for i in range(len(filtered_sonnets)):
    clean_sonnets.append(dctk.clean_data(filtered_sonnets[i]))

In [15]:
# much better!
clean_sonnets

[' ',
 ' ',
 'f',
 'r',
 'o',
 'm',
 ' ',
 'f',
 'a',
 'i',
 'r',
 'e',
 's',
 't',
 ' ',
 'c',
 'r',
 'e',
 'a',
 't',
 'u',
 'r',
 'e',
 's',
 ' ',
 'w',
 'e',
 ' ',
 'd',
 'e',
 's',
 'i',
 'r',
 'e',
 ' ',
 'i',
 'n',
 'c',
 'r',
 'e',
 'a',
 's',
 'e',
 '',
 ' ',
 ' ',
 't',
 'h',
 'a',
 't',
 ' ',
 't',
 'h',
 'e',
 'r',
 'e',
 'b',
 'y',
 ' ',
 'b',
 'e',
 'a',
 'u',
 't',
 'y',
 '',
 's',
 ' ',
 'r',
 'o',
 's',
 'e',
 ' ',
 'm',
 'i',
 'g',
 'h',
 't',
 ' ',
 'n',
 'e',
 'v',
 'e',
 'r',
 ' ',
 'd',
 'i',
 'e',
 '',
 ' ',
 ' ',
 'b',
 'u',
 't',
 ' ',
 'a',
 's',
 ' ',
 't',
 'h',
 'e',
 ' ',
 'r',
 'i',
 'p',
 'e',
 'r',
 ' ',
 's',
 'h',
 'o',
 'u',
 'l',
 'd',
 ' ',
 'b',
 'y',
 ' ',
 't',
 'i',
 'm',
 'e',
 ' ',
 'd',
 'e',
 'c',
 'e',
 'a',
 's',
 'e',
 '',
 ' ',
 ' ',
 'h',
 'i',
 's',
 ' ',
 't',
 'e',
 'n',
 'd',
 'e',
 'r',
 ' ',
 'h',
 'e',
 'i',
 'r',
 ' ',
 'm',
 'i',
 'g',
 'h',
 't',
 ' ',
 'b',
 'e',
 'a',
 'r',
 ' ',
 'h',
 'i',
 's',
 ' ',
 'm',
 'e',
 'm',
 '

### Use your data tool to create character sequences 

We'll need the `create_char_sequenes` method for this task. However this method requires a parameter call `maxlen` which is responsible for setting the maximum sequence length. 

So what would be a good sequence length, exactly? 

In order to answer that question, let's do some statistics! 

In [16]:
def calc_stats(corpus):
    """
    Calculates statisics on the length of every line in the sonnets
    """
    
    # write a list comprehension that calculates each sonnets line length - save the results to `doc_lens` 
    doc_lens = [len(i) for i in corpus]

    # use numpy to calcualte and return the mean, median, std, max, min of the doc lens - all in one line of code

    return np.mean(doc_lens), np.median(doc_lens), np.std(doc_lens), np.max(doc_lens), np.min(doc_lens)

In [17]:
# sonnet line length statistics 
mean ,med, std, max_, min_ = calc_stats(clean_sonnets)
mean, med, std, max_, min_ 

(0.9622570006350247, 1.0, 0.19057404168435638, 1, 0)

In [18]:
# using the results of the sonnet line length statistics, use your judgement and select a value for maxlen
# use .create_char_sequences() to create sequences

# YOUR CODE HERE
maxlen = 49
dctk.create_char_sequenes(clean_sonnets, maxlen=maxlen, step=5)

sequences:  38307


Take a look at the `data_cleaning_toolkit_class.py` file. 

In the first 4 lines of code in the `create_char_sequences` method, class attributes `n_features` and `unique_chars` are created. Let's call them in the cells below.

In [19]:
# number of input features for our LSTM model
dctk.n_features

27

In [20]:
# unique charactes that appear in our sonnets 
dctk.chars

['e',
 'q',
 'x',
 'p',
 'i',
 'f',
 'h',
 'u',
 'w',
 ' ',
 'n',
 'v',
 'j',
 'z',
 'y',
 'm',
 'k',
 't',
 'o',
 'r',
 'b',
 'l',
 'g',
 'a',
 'd',
 's',
 'c']

## Time for Questions 

----
**Question 1:** 

Why are the `number of unique characters` (i.e. **dctk.unique_chars**) and the `number of model input features` (i.e. **dctk.n_features**) the same?

**Hint:** The model that we will shortly be building here is very similar to the text generation model that we built in the guided project.

**Answer 1:**

The number of unique character and n_features is the same i.e. 23


**Question 2:**

Take a look at the print out of `dctk.unique_chars` one more time. Notice that there is a white space. 

Why is it desirable to have a white space as a possible character to predict?

**Answer 2:**

A white space makes a sentence readable else there will not be any separator between words to make sense from.

----

### Use our data tool to create X and Y splits

You'll need the `create_X_and_Y` method for this task. 

In [21]:
# TODO: provide a walk through of data_cleaning_toolkit with unit tests that check for understanding 
X, y = dctk.create_X_and_Y()

![](https://miro.medium.com/max/891/0*jGB1CGQ9HdeUwlgB)

In [22]:
# notice that our input matrix isn't actually a matrix - it's a rank 3 tensor
X.shape

(38307, 49, 27)

In $X$.shape we see three numbers (*n1*, *n2*, *n3*). What do these numbers mean?

Well, *n1* tells us the number of samples that we have. But what about the other two?

In [23]:
# first index returns a signle sample, which we can see is a sequence 
first_sample_index = 0 
X[first_sample_index]

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False,  True, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

Notice that each sequence (i.e. $X[i]$ where $i$ is some index value) is `maxlen` long and has `dctk.n_features` number of features. Let's try to better understand this shape. 

In [24]:
# each sequence is maxlen long and has dctk.n_features number of features
X[first_sample_index].shape

(49, 27)

**Each row corresponds to a character vector** and there are `maxlen` number of character vectors. 

**Each column corresponds to a unique character** and there are `dctk.n_features` number of features. 


In [25]:
# let's index for a single character vector 
first_char_vect_index = 0
X[first_sample_index][first_char_vect_index]

array([False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

Notice that there is a single `TRUE` value and all the rest of the values are `FALSE`. 

This is a one-hot encoding for which character appears at each index within a sequence. Specifically, the cell above is looking at the first character in the sequence.

Only a single character can appear as the first character in a sequence, so there will necessarily be a single `TRUE` value and the rest will be `FALSE`. 

Let's say that `TRUE` appears in the $ith$ index; by  $ith$ index we simply mean some index in the general case. How can we find out which character that actually corresponds to?

To answer this question, we need to use the character-to-integer look up dictionaries. 

In [26]:
# take a look at the index to character dictionary
# if a TRUE appears in the 0th index of a character vector,
# then we know that whatever char you see below next to the 0th key 
# is the character that that character vector is endcoding for
dctk.int_char

{0: 'e',
 1: 'q',
 2: 'x',
 3: 'p',
 4: 'i',
 5: 'f',
 6: 'h',
 7: 'u',
 8: 'w',
 9: ' ',
 10: 'n',
 11: 'v',
 12: 'j',
 13: 'z',
 14: 'y',
 15: 'm',
 16: 'k',
 17: 't',
 18: 'o',
 19: 'r',
 20: 'b',
 21: 'l',
 22: 'g',
 23: 'a',
 24: 'd',
 25: 's',
 26: 'c'}

In [27]:
# let's look at an example to tie it all together

seq_len_counter = 0

# index for a single sample 
for seq_of_char_vects in X[first_sample_index]:
    
    # get index with max value, which will be the one TRUE value 
    index_with_TRUE_val = np.argmax(seq_of_char_vects)
    
    print (dctk.int_char[index_with_TRUE_val])
    
    seq_len_counter+=1
    
print ("Sequence length: {}".format(seq_len_counter))

 
 
 
 
f
 
r
 
o
 
m
 
 
 
f
 
a
 
i
 
r
 
e
 
s
 
t
 
 
 
c
 
r
 
e
 
a
 
t
 
u
 
r
 
e
 
s
 
 
Sequence length: 49


## Time for Questions 

----
**Question 1:** 

In your own words, how would you describe the numbers from the shape print out of `X.shape` to a fellow classmate?


**Answer 1:**

The X.shape gives three numbers. The first number is the number of sample easy.
The second number is the maxlength after which we have truncated each sentence. This is simply the row representation of the original row in True or False.
The third number is the feature matrix or column that is unique and correspond to 23 unique characters.

----


### Build a Text Generation model

Now that we have prepped our data (and understood that process) let's finally build out our character generation model, similar to what we did in the guided project.

In [28]:
def sample(preds, temperature=1.0):
    """
    Helper function to sample an index from a probability array
    """
    # convert preds to array 
    preds = np.asarray(preds).astype('float64')
    # scale values 
    preds = np.log(preds) / temperature
    # exponentiate values
    exp_preds = np.exp(preds)
    # this equation should look familar to you (hint: it's an activation function)
    preds = exp_preds / np.sum(exp_preds)
    # Draw samples from a multinomial distribution
    probas = np.random.multinomial(1, preds, 1)
    # return the index that corresponds to the max probability 
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    """"
    Function invoked at end of each epoch. Prints the text generated by our model.
    """
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    

    # randomly pick a starting index 
    # will be used to take a random sequence of chars from `text`
    start_index = random.randint(0, len(text) - dctk.maxlen - 1)
    
    # this is our seed string (i.e. input seqeunce into the model)
    generated = ''

    # start the sentence at index `start_index` and include the next` dctk.maxlen` number of chars
    sentence = text[start_index: start_index + dctk.maxlen]

    # add to generated
    generated += sentence

    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    # use model to predict what the next 40 chars should be that follow the seed string
    for i in range(40):

        # shape of a single sample in a rank 3 tensor 
        x_dims = (1, dctk.maxlen, dctk.n_features)
        # create an array of zeros with shape x_dims
        # recall that python considers zeros and boolean FALSE as the same
        x_pred = np.zeros(x_dims)

        # create a seq vector for our randomly select sequence 
        # i.e. create a numerical encoding for each char in the sequence 
        for t, char in enumerate(sentence):
            # for sample 0 in seq index t and character `char` encode a 1 (which is the same as a TRUE)
            x_pred[0, t, dctk.char_int[char]] = 1

        # next, take the seq vector and pass into model to get a prediction of what the next char should be 
        preds = model.predict(x_pred, verbose=0)[0]
        # use the sample helper function to get index for next char 
        next_index = sample(preds)
        # use look up dict to get next char 
        next_char = dctk.int_char[next_index]

        # append next char to sequence 
        sentence = sentence[1:] + next_char 
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [29]:
# need this for on_epoch_end()
text = " ".join(clean_sonnets)

In [30]:
# create callback object that will print out text generation at the end of each epoch 
# use for real-time monitoring of model performance
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

----
### Train Model

Build a text generation model using LSTMs. Feel free to reference the model used in the guided project. 

It is recommended that you train this model to at least 50 epochs (but more if you're computer can handle it). 

You are free to change up the architecture as you wish. 

Just in case you have difficultly training a model, there is a pre-trained model saved to a file called `trained_text_gen_model.h5` that you can load in (the same way that you learned how to load in Keras models in Sprint 2 Module 4). 

In [31]:
!pip install -U numpy==1.18.5

  Using cached numpy-1.18.5-cp37-cp37m-macosx_10_9_x86_64.whl (15.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 1.18.5 which is incompatible.


In [32]:
np.__version__

'1.19.5'

In [40]:
import numpy as np
# build text generation model layer by layer
model = Sequential()
model.add(Embedding(input_dim=23, output_dim=128, input_length=maxlen))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))

model.add(Dense(1, activation='sigmoid'))

# fit model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()
# YOUR CODE HERE

model.fit(X, y, batch_size=32, epochs=50, verbose=1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 49, 128)           2944      
_________________________________________________________________
lstm_2 (LSTM)                (None, 49, 128)           131584    
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 266,241
Trainable params: 266,241
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50


ValueError: in user code:

    /Users/rob/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:855 train_function  *
        return step_function(self, iterator)
    /Users/rob/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/rob/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/rob/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/rob/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3608 _call_for_each_replica
        return fn(

In [34]:
# save trained model to file 
model.save("trained_text_gen_model.h5")

### Let's play with our trained model 

Now that we have a trained model that, though far from perfect, is able to generate actual English words, we can take a look at the predictions to continue to learn more about how a text generation model works. 

We can also take this as an opportunity to unpack the `def on_epoch_end` function to better understand how it works. 

In [35]:
# this is our joinned clean sonnet data
text

'    f r o m   f a i r e s t   c r e a t u r e s   w e   d e s i r e   i n c r e a s e      t h a t   t h e r e b y   b e a u t y  s   r o s e   m i g h t   n e v e r   d i e      b u t   a s   t h e   r i p e r   s h o u l d   b y   t i m e   d e c e a s e      h i s   t e n d e r   h e i r   m i g h t   b e a r   h i s   m e m o r y      b u t   t h o u    c o n t r a c t e d   t o   t h i n e   o w n   b r i g h t   e y e s      f e e d  s t   t h y   l i g h t  s   f l a m e   w i t h   s e l f  s u b s t a n t i a l   f u e l      m a k i n g   a   f a m i n e   w h e r e   a b u n d a n c e   l i e s      t h y   s e l f   t h y   f o e    t o   t h y   s w e e t   s e l f   t o o   c r u e l      t h o u   t h a t   a r t   n o w   t h e   w o r l d  s   f r e s h   o r n a m e n t      a n d   o n l y   h e r a l d   t o   t h e   g a u d y   s p r i n g      w i t h i n   t h i n e   o w n   b u d   b u r i e s t   t h y   c o n t e n t      a n d   t e n d e r   c h u r l   m

In [36]:
# randomly pick a starting index 
# will be used to take a random sequence of chars from `text`
# run this cell a few times and you'll see `start_index` is random
start_index = random.randint(0, len(text) - dctk.maxlen - 1)
start_index

98128

In [37]:
# next use the randomly selected starting index to sample a sequence from the `text`

# this is our seed string (i.e. input seqeunce into the model)
generated = ''

# start the sentence at index `start_index` and include the next` dctk.maxlen` number of chars
sentence = text[start_index: start_index + dctk.maxlen]

# add to generated
generated += sentence

generated

' b e   c a s t   a w a y          t h e   w o r s'

In [38]:
# this block of code let's us know what the seed string is 
# i.e. the input seqeunce into the model
print('----- Generating with seed: "' + sentence + '"')
sys.stdout.write(generated)

----- Generating with seed: " b e   c a s t   a w a y          t h e   w o r s"
 b e   c a s t   a w a y          t h e   w o r s

In [39]:
x_pred.shape

NameError: name 'x_pred' is not defined

In [ ]:
# use model to predict what the next 40 chars should be that follow the seed string
for i in range(40):

    # shape of a single sample in a rank 3 tensor 
    x_dims = (1, dctk.maxlen, dctk.n_features)
    # create an array of zeros with shape x_dims
    # recall that python considers zeros and boolean FALSE as the same
    x_pred = np.zeros(x_dims)

    # create a seq vector for our randomly select sequence 
    # i.e. create a numerical encoding for each char in the sequence 
    for t, char in enumerate(sentence):
        # for sample 0 in seq index t and character `char` encode a 1 (which is the same as a TRUE)
        x_pred[0, t, dctk.char_int[char]] = 1

    # next, take the seq vector and pass into model to get a prediction of what the next char should be 
    preds = model.predict(x_pred, verbose=0)[0]
    # use the sample helper function to get index for next char 
    next_index = sample(preds)
    # use look up dict to get next char 
    next_char = dctk.int_char[next_index]

    # append next char to sequence 
    sentence = sentence[1:] + next_char 

In [ ]:
# this is the seed string
generated

In [ ]:
# these are the 40 chars that the model thinks should come after the seed stirng
sentence

In [ ]:
# how put it all together
generated + sentence

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN